In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /content/. Or use the environment method.


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [4]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [5]:
!unzip -q train.zip -d .

In [ ]:
# 파일 개수
import os
print(len(os.listdir('/content/train/')))

In [ ]:
!rm -r '/content/dataset/'

In [8]:
!mkdir '/content/dataset/'
!mkdir '/content/dataset/cat/'
!mkdir '/content/dataset/dog/'

In [9]:
import shutil

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/cat/'+i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/dog/'+i)

In [11]:
import tensorflow as tf
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset="training",
    validation_split=0.2,
    seed=1234
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset="validation",
    validation_split=0.2,
    seed=1234
)

def preproFun(i,result):
  i = tf.cast( i/255.0, tf.float32 )
  return i,result

train_ds = train_ds.map(preproFun)
val_ds = val_ds.map(preproFun)
import matplotlib.pyplot as plt

# for i,정답 in train_ds.take(1):
#   print(i)
#   print(정답)
#   plt.imshow(i[0].numpy().astype('uint8'))
#   plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [14]:
model = tf.keras.Sequential([
    
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds, epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_2 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_rotation (RandomRota  (None, 64, 64, 3)        0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1